In [3]:
import pandas as pd

from Bio import SeqIO

import os
from Bio.Blast.Applications import NcbiblastpCommandline

import seaborn as sns

In [4]:
data = pd.read_csv('../combined_data.csv')
data['autoimmune'] = data['category'] == 'autoimmune'

In [8]:
data

,fold,category,id,gene,pe_level,sequence,autoimmune
0,4,autoimmune,A0A075B6H9,IGLV4-69,1,MAWTPLLFLTLLLHCTGSLSQLVLTQSPSASASLGASVKLTCTLSS...,True
1,3,autoimmune,A0A075B6N2,TRBV20-1,3,MLLLLLLLGPGSGLGAVVSQHPSRVICKSGTSVKIECRSLDFQATT...,True
2,4,autoimmune,A0A087WZ39,TRBV15,1,MGPGLLHWMALCLLGTGHGDAMVIQNPRYQVTQFGKPVTLSCSQTL...,True
3,4,autoimmune,A0A0A0MS06,TRBV23-1,1,MGTRLLGCAALCLLAADSFHAKVTQTPGHLVKGKGQKTKMDCTPEK...,True
4,3,autoimmune,A0A0G2JHM8,MOG,1,MASLSRPSLPSCLCSFLLLLLLQVSSSYAGQFRVIGPRHPIRALVG...,True
...,...,...,...,...,...,...,...
20643,4,non_autoimmune,P0DMU3,NaN,3,MVSSKGLWKERPSAHTSECFSTTACPVAFILLVWNSQTPAGLQSLC...,False
20644,4,non_autoimmune,Q6ZSN1,NaN,2,MGVPRAREGRGAGSQSPPRGRCLHPFRWGSQDRGRGEGLALSPLLP...,False
20645,2,non_autoimmune,Q96M85,NaN,2,MSHSRRAAPTQDQCHTPGFPTSRETSGSIWQARICGSLQALDTWRT...,False
20646,2,non_autoimmune,Q96NJ1,NaN,2,MRRERPELRDAEGRLRLRAGCLVTAWPRAPSGAGSWSMAAASPWPA...,False
